# Exemplo notebook PySpark com sessão Glue
- https://aws.amazon.com/blogs/machine-learning/prepare-data-at-scale-in-amazon-sagemaker-studio-using-serverless-aws-glue-interactive-sessions/

## Comandos Sparkmagic para configurar a sessão do Glue

In [5]:
%help


Available Magic Commands

## Sessions Magics
%help | Return a list of descriptions and input types for all magic commands. 
%profile | String | Specify a profile in your aws configuration to use as the credentials provider.
%region | String | Specify the AWS region in which to initialize a session | Default from ~/.aws/configure
%idle_timeout | Int | The number of minutes of inactivity after which a session will timeout. The default idle timeout value is 2880 minutes (48 hours).
%session_id | Returns the session ID for the running session. 
%session_id_prefix | String | Define a String that will precede all session IDs in the format [session_id_prefix]-[session_id]. If a session ID is not provided, a random UUID will be generated.
%status | Returns the status of the current Glue session including its duration, configuration and executing user / role.
%list_sessions | Lists all currently running sessions by name and ID.
%stop_session | Stops the current session.
%glue_version | String 

In [11]:
%stop_session
%session_id_prefix pyspark-sample-
%glue_version 3.0
%idle_timeout 60
%%configure
{
"--enable-spark-ui": "true",
"--spark-event-logs-path": "s3://sm-bucket-16600123247018237842/gis-spark-logs/"
}

Stopping session: 0aeac2e3-cd81-4313-8534-cd253dc7b598
Stopped session.
Setting session ID prefix to pyspark-sample-
Setting Glue version to: 3.0
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 60 minutes.
The following configurations have been updated: {'--enable-spark-ui': 'true', '--spark-event-logs-path': 's3://sm-bucket-16600123247018237842/gis-spark-logs/'}


## Exemplos comandos PySpark

### Iniciando sessão Spark com Glue

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::163733418386:role/sm_user_role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: b408a6e7-8bfd-4b9f-bf2b-a0488e098ca8
Applying the following default arguments:
--glue_kernel_version 0.36
--enable-glue-datacatalog true
--enable-spark-ui true
--spark-event-logs-path s3://sm-bucket-16600123247018237842/gis-spark-logs/
Waiting for session b408a6e7-8bfd-4b9f-bf2b-a0488e098ca8 to get into ready status...
Session b408a6e7-8bfd-4b9f-bf2b-a0488e098ca8 has been created




### Executando comandos SQL com Spark

In [2]:
spark.sql("show databases").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|              demodb|
|sagemaker_processing|
+--------------------+


In [3]:
spark.sql("use demodb").show()
spark.sql("show tables").show()

++
||
++
++

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|  demodb|analytics_titanic|      false|
|  demodb|     clickstreams|      false|
|  demodb|          courses|      false|
+--------+-----------------+-----------+


In [4]:
spark.sql("select * from analytics_titanic limit 10").show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|passengerid|survived|pclass|                name|   sex| age|sibsp|parch|          ticket|   fare|cabin|embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|       null|    null|  null|                Name|   Sex|null| null| null|          Ticket|   Fare|Cabin|Embarked|
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|     |       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|712.833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|     |       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|      

### Criando DynamicFrame de uma tabela do Glue
- https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-dynamic-frame.html

In [5]:
dyf = glueContext.create_dynamic_frame.from_catalog(database="demodb", table_name="analytics_titanic")
dyf.printSchema()

root
|-- passengerid: long
|-- survived: long
|-- pclass: long
|-- name: string
|-- sex: string
|-- age: double
|-- sibsp: long
|-- parch: long
|-- ticket: string
|-- fare: string
|-- cabin: string
|-- embarked: string


### Convertendo DynamicFrame para Dataframe

In [6]:
df = dyf.toDF()
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|passengerid|survived|pclass|                name|   sex| age|sibsp|parch|          ticket|   fare|cabin|embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|     |       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|712.833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|     |       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|     |       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

### Usando funções PySpark
- https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [7]:
from pyspark.sql.functions import lower

In [8]:
df_name = df.filter(lower((df.name)).contains('braund'))
df_name.show()

+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|passengerid|survived|pclass|                name| sex| age|sibsp|parch|   ticket|  fare|cabin|embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|male|22.0|    1|    0|A/5 21171|  7.25|     |       S|
|        478|       0|     3|Braund, Mr. Lewis...|male|29.0|    1|    0|     3460|70.458|     |       S|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+


### Convertendo DataFrame para DynamicFrame

In [12]:
from awsglue.dynamicframe import DynamicFrame

In [13]:
dyf_name = DynamicFrame.fromDF(df_name, glueContext, 'name_titanic')

### Armazenando  o DynamicFrame no S3  e atualizando o catálogo do Glue

In [14]:
s3output = glueContext.getSink(
  path="s3://sm-bucket-16600123247018237842/titanic",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)

s3output.setCatalogInfo(
  catalogDatabase="demodb", catalogTableName="name_titanic"
)

s3output.setFormat("glueparquet")
s3output.writeFrame(dyf_name)

In [15]:
spark.sql("select * from name_titanic").show()

+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|passengerid|survived|pclass|                name| sex| age|sibsp|parch|   ticket|  fare|cabin|embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|male|22.0|    1|    0|A/5 21171|  7.25|     |       S|
|        478|       0|     3|Braund, Mr. Lewis...|male|29.0|    1|    0|     3460|70.458|     |       S|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
